## Code for the discussed methods
- Q-learning

In [ ]:
import numpy as np
import gym
from tqdm import tqdm
import pickle
import random

### Monte-Carlo

In [ ]:
env = gym.make('FrozenLake-v0')
s0 = env.reset()
env.render()
# when you make action there is 33% chance to get to another space
# LEFT = 0
# DOWN = 1
# RIGHT = 2
# UP = 3

In [ ]:
def e_greedy_policy(q_table, state, k, env):
    #epsilon = 0.25
    epsilon = 1 - (k/number_of_runs)
    actions_count = 4
    act_greedy = epsilon / actions_count + (1 - epsilon)
    
    if random.random() < act_greedy:
        #action = np.argmax(q_table, axis=1)[state]
        action = np.argmax(q_table[state,:])
    else:
        action = env.action_space.sample()
        
    return action

def random_policy(q_table, state, k, env):
    action = env.action_space.sample()
    return action

In [ ]:
state_values = np.zeros(env.observation_space.n)
n_of_s = np.zeros(env.observation_space.n)

q_table = np.zeros((env.observation_space.n, env.action_space.n))
#q_table = np.random.random((env.observation_space.n, env.action_space.n))
q_table_n = np.zeros((env.observation_space.n, env.action_space.n))

number_of_runs = 500000
number_of_won = 0
for k in tqdm(range(1, number_of_runs)):
    trajectory_states = []
    trajectory_actions = []
    trajectory_rewards = []
    trajectory_total_discounted_rewards = []
    
    current_state = env.reset()
    done = False
    won = False
    while not done: 
        action = e_greedy_policy(q_table, current_state, k, env)
        new_state, reward, done, prob = env.step(action)
        
        trajectory_states.append(current_state)
        trajectory_actions.append(action)
        trajectory_rewards.append(reward)
        
        current_state = new_state
        if done:
            #env.render()
            if reward == 1:
                won = True
                
    for idx_state, state in enumerate(trajectory_states):
        total_discounted_reward = 0
        gama = 0.9
        for idx_reward, reward in enumerate(trajectory_rewards[idx_state:]):
            total_discounted_reward += reward * (gama**idx_reward)
        trajectory_total_discounted_rewards.append(total_discounted_reward)

    # fill action states values
    for step in range(len(trajectory_states)):
        step_state = trajectory_states[step]
        step_action = trajectory_actions[step]
        q_table_n[step_state][step_action] += 1
        n = q_table_n[step_state][step_action]

        current_value = q_table[step_state][step_action]
        new_value = current_value + (1/n) * (trajectory_total_discounted_rewards[step] - current_value)
        q_table[step_state][step_action] = new_value

    # fill states values
    for step, state in enumerate(trajectory_states):
        n_of_s[state] += 1 
        state_values[state] = state_values[state] + (1/n_of_s[state]) * (trajectory_total_discounted_rewards[step]-state_values[state])

        
    if won:
#         print(trajectory_states)
#         print(trajectory_actions)
#         print(trajectory_total_discounted_rewards)
#         print(q_table)
        number_of_won += 1
        

if number_of_runs >= 100000:
    with open(str(number_of_runs) + '_data.pickle', 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(q_table, f, pickle.HIGHEST_PROTOCOL)

    with open(str(number_of_runs) + '_data.pickle', 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
        print(data)
        
print(state_values.reshape(4,4))

In [ ]:
def get_super_q_table():
    number_of_runs = 1000000
    with open(str(number_of_runs) + '_data.pickle', 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
        return data
print(get_super_q_table())
print(q_table)
print(q_table_n)

def get_action(q_table, s1):
    action = np.argmax(q_table, axis=1)[s1]
    #action = env.action_space.sample()
    return action

def evaluate_policy(env, q_table, max_episodes=1000): 
    tot_reward = 0
    for ep in range(max_episodes):
        state = env.reset()
        done = False
        ep_reward = 0
        
        # Reward per episode
        while not done:
            action = get_action(q_table, state)
            new_state, reward, done, _ = env.step(action)
            ep_reward += reward
            state = new_state
            if done:
                tot_reward += ep_reward
    return tot_reward / max_episodes

print('won', number_of_won)
print('Result', evaluate_policy(env, q_table, 1000))

In [ ]:
class RandomGuessPolicy(object):
    def __init__(self, env):
        self.env = env

    def what_to_do(self, state):
        action = self.env.action_space.sample()
        return action

In [ ]:
for n in range(5):
    policy = RandomGuessPolicy(gym.make('FrozenLake-v0'))
    print(policy.what_to_do(n))
    
    # train
    
    # evaluate